# Imports

In [1]:
import pm4py
import pandas as pd
from TreeToTptp import TreeToTptp
from ProverWrapper import use_provers
from create_thesis_with_2_spec import create_thesis_with_2_spec, LogicFunctions
import os

# Helpers

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import graphviz
import os


def create_tree_graph(process_tree, output_file="process_tree"):
    if not output_file.endswith('.png'):
        output_file = output_file + '.png'

    dot = graphviz.Digraph()
    dot.attr(rankdir='TB')

    def add_nodes_edges(tree, parent_id=None, counter=[0]):
        current_id = str(counter[0])
        counter[0] += 1

        if tree.children:
            label = str(tree.operator)
            label = 'seq' if label == '->' else 'xor' if label == 'X' else 'loop' if label == '*' else 'and' if label == '+' else label
            dot.node(current_id, label, shape='box')
        else:
            if tree.label is None:
                dot.node(current_id, "", shape='circle',
                         style='filled', fillcolor='black', width='0.3')
            else:
                dot.node(current_id, tree.label, shape='oval')

        if parent_id is not None:
            dot.edge(str(parent_id), current_id)

        for child in tree.children:
            add_nodes_edges(child, int(current_id), counter)

    add_nodes_edges(process_tree)
    dot.render(output_file[:-4], format='png', cleanup=True)

    if not os.path.exists(output_file):
        raise FileNotFoundError(f"Failed to create image file: {output_file}")

    return output_file

In [3]:
def get_vampire_short(vampire: str):
    return ' '.join([l for l in vampire.splitlines() if l.startswith('% Refutation') or l.startswith('% SZS status') or l.startswith('% Termination reason')])

def get_eprover_short(eprover: str):
    return ' '.join([l for l in eprover.splitlines() if l.startswith('# SZS status') or l.startswith('# Proof found') or l.startswith('# No proof found')])

In [4]:
from Julia_Origin import ProcessTreeAdapter, get_pattern_expression, get_results
from converter import convert_to_tptp

def use_julias_code(tree):
    W1 = str(tree)

    W1 = ProcessTreeAdapter.remove_brackets_between_single_quotes(W1)

    W1 = ProcessTreeAdapter.replace_spaces_with_underscore(W1)
    W1 = W1.replace("( ", "(").replace(") ", ")").replace(" )",
                                                          ")").replace("->", ">")

    labelled_pattern_expression = ProcessTreeAdapter.label_expressions(W1)

    pattern_expression = get_pattern_expression(labelled_pattern_expression)

    results = get_results(pattern_expression)[:-1]
    results = [x.rstrip() for x in results.splitlines()]
    tptp = convert_to_tptp(results)
    return pattern_expression, '\n'.join(results), tptp

In [5]:
def run_satisfiability(folder_name, log, threshold, prefix='', use_julia=False):
    try:
        os.makedirs(folder_name, exist_ok=True)
        
        process_tree = pm4py.discover_process_tree_inductive(
            log, threshold, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
        
        if use_julia:
            expr, spec, tptp = use_julias_code(process_tree)
        else:
            expr, spec, tptp = TreeToTptp(prefix).tree_to_tptp(process_tree)
        
        with open(f'{folder_name}/expression.txt', 'w') as f:
            f.write(expr)
        with open(f'{folder_name}/specification.txt', 'w') as f:
            f.write(spec)
        with open(f'{folder_name}/tptp.p', 'w') as f:
            f.write(tptp)

        vampire, eprover = use_provers(f'{folder_name}/tptp.p')
        with open(f'{folder_name}/vampire.txt', 'w') as f:
            f.write(vampire)
        with open(f'{folder_name}/eprover.txt', 'w') as f:
            f.write(eprover)

        create_tree_graph(process_tree, f'{folder_name}/process_tree.png')

        return folder_name, get_vampire_short(vampire), get_eprover_short(eprover)
    except Exception as e:
        print(e)
        return folder_name, f'Error: {e}', f'Error: {e}'


def run_comparison(folder_name, log, threshold1, threshold2, logic: LogicFunctions, use_julia=False):
    try:
        os.makedirs(folder_name, exist_ok=True)
        
        run_satisfiability(f'{folder_name}/original1', log, threshold1, 'A', use_julia)
        run_satisfiability(f'{folder_name}/original2', log, threshold2, 'B', use_julia)

        result = create_thesis_with_2_spec(f'{folder_name}/original1/tptp.p', f'{folder_name}/original2/tptp.p', logic)
        with open(f'{folder_name}/tptp_{logic.name}.p', 'w') as f:
            f.write(result)

        vampire, eprover = use_provers(f'{folder_name}/tptp_{logic.name}.p')
        with open(f'{folder_name}/vampire_{logic.name}.txt', 'w') as f:
            f.write(vampire)
        with open(f'{folder_name}/eprover_{logic.name}.txt', 'w') as f:
            f.write(eprover)

        return folder_name, get_vampire_short(vampire), get_eprover_short(eprover)
    except Exception as e:
        print(e)
        return folder_name, f'Error: {e}', f'Error: {e}'


def run_thesis(folder_name, log, threshold, thesis, use_julia=False):
    try:
        os.makedirs(folder_name, exist_ok=True)
        
        run_satisfiability(f'{folder_name}/original', log, threshold, use_julia=use_julia)
        with open(f'{folder_name}/tptp_thesis.p', 'w') as f:
            with open(f'{folder_name}/original/tptp.p', 'r') as f2:
                tptp = f2.read()
                f.write(f'{tptp}\n\n{thesis}')

        vampire, eprover = use_provers(f'{folder_name}/tptp_thesis.p')
        with open(f'{folder_name}/vampire_thesis.txt', 'w') as f:
            f.write(vampire)
        with open(f'{folder_name}/eprover_thesis.txt', 'w') as f:
            f.write(eprover)
            
        return folder_name, get_vampire_short(vampire), get_eprover_short(eprover)
    except Exception as e:
        print(e)
        return folder_name, f'Error: {e}', f'Error: {e}'

# Julia's problems

## logs 

In [6]:
running = pm4py.read_xes('Data/running-example.xes')

/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 6/6 [00:00<00:00, 2602.73it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/lo

In [7]:
repair = pm4py.format_dataframe(pd.read_csv("Data/repairExample.csv", sep=','),
                                case_id='Case ID', activity_key='Activity', timestamp_key='Start Timestamp')

/tmp/ipykernel_203240/2258410974.py:1: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  repair = pm4py.format_dataframe(pd.read_csv("Data/repairExample.csv", sep=','),
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13

## Theorems

In [8]:
running_thesis = [
    '''
fof(thesis, conjecture, (
    ! [X]: (register_request(X) => ?[Y]: (reject_request(Y) | pay_compensation(Y)))
)).
''',
    '''
fof(thesis, conjecture, (
    ! [X]: (~(reject_request(X) & pay_compensation(X)))
)).
'''
]

repair_thesis = [
    '''
fof(thesis, conjecture, (
    ! [X]: (register(X) => ?[Y]: (repair_simple(Y) | repair_complex(Y)))
)).
''',
    '''
fof(thesis, conjecture, (
    ! [X]: (~(inform_user(X) & tau(X)))
)).
'''
]

## Julia's code

### Code

In [9]:
julias_results = [
    run_satisfiability('julia_s_problems/problem1',
                       running, 0.0, use_julia=True),
    run_thesis('julia_s_problems/problem11', running,
               0.0, running_thesis[0], use_julia=True),
    run_thesis('julia_s_problems/problem12', running,
               0.0, running_thesis[1], use_julia=True),

    run_satisfiability('julia_s_problems/problem2',
                       repair, 0.0, use_julia=True),
    run_satisfiability('julia_s_problems/problem3',
                       repair, 0.5, use_julia=True),
    run_satisfiability('julia_s_problems/problem4',
                       repair, 1.0, use_julia=True),
    run_comparison('julia_s_problems/problem5', repair,
                   0.0, 0.5, LogicFunctions.IMPLIES, use_julia=True),
    run_comparison('julia_s_problems/problem6', repair,
                   0.5, 1.0, LogicFunctions.IMPLIES, use_julia=True),
    run_comparison('julia_s_problems/problem8', repair,
                   0.0, 0.5, LogicFunctions.EQUIVALENT, use_julia=True),
    run_comparison('julia_s_problems/problem9', repair,
                   0.5, 1.0, LogicFunctions.EQUIVALENT, use_julia=True),
    run_thesis('julia_s_problems/problem13', repair,
               0.5, repair_thesis[0], use_julia=True),
    run_thesis('julia_s_problems/problem14', repair,
               0.5, repair_thesis[1], use_julia=True),
    run_thesis('julia_s_problems/problem15', repair,
               1.0, repair_thesis[0], use_julia=True),
    run_thesis('julia_s_problems/problem16', repair,
               1.0, repair_thesis[1], use_julia=True),
]

Seq3(1]register_request, Loop(2]l_s,Seq2(3]And2(4]a2_s,check_ticket, Xor2(5]x2_s,examine_casually, examine_thoroughly,x2_e[5),a2_e[4), decide[3), reinitiate_request[2), Xor2(2]x2_s,pay_compensation, reject_request,x2_e[2)[1)
ini: register_request
fin: x2_e

Wynik: 
ForAll((a2_s | a2_e) => Exist(decide))
ForAll(~((x2_s) ^ (examine_casually)))
ForAll(~((a2_s) ^ ((check_ticket) | (x2_s | x2_e))))
ForAll((l_s) => Exist(a2_s | decide))
ForAll((register_request) => Exist(l_s | decide))
ForAll(~((pay_compensation) ^ (reject_request)))
ForAll(~((a2_s | a2_e) ^ (decide)))
ForAll(~((x2_s) ^ (examine_thoroughly)))
ForAll((a2_s) => (Exist(check_ticket) ^ Exist(x2_s | x2_e)))
ForAll((x2_s | x2_e) => Exist(a2_e))
ForAll(~((x2_s) ^ (pay_compensation)))
ForAll((check_ticket) => Exist(a2_e))
ForAll((x2_s) => ((Exist(pay_compensation) ^ ~(Exist(reject_request))) | (~(Exist(pay_compensation)) ^ Exist(reject_request))))
Exist(x2_s)
ForAll(((examine_casually) | (examine_thoroughly)) => Exist(x2_e))
ForAll(

### Readable output

In [10]:
for r in julias_results:
    print(r[0])
    print('  Vampire:')
    for i in r[1].split('%')[1:]:
        print(f'    %{i}')
    print('  Eprover:')
    for i in r[2].split('#')[1:]:
        print(f'    %{i}')
    print('\n')

julia_s_problems/problem1
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status Satisfiable


julia_s_problems/problem11
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_thesis 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Theorem


julia_s_problems/problem12
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_thesis 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Theorem


julia_s_problems/problem2
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Unsatisfiable for tptp 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Unsatisfiable


julia_s_problems/problem3
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status S

### Csv

In [11]:
julia_s_csv = '\n'.join([f'{r[0]},{r[1]},{r[2]}' for r in julias_results])
with open('julia_s_problems/julia_s_results.csv', 'w') as f:
    f.write(julia_s_csv)

## Our results

### Code

In [12]:
our_results = [
    run_satisfiability('our_problems/problem1',
                       running, 0.0),
    run_thesis('our_problems/problem11', running,
               0.0, running_thesis[0]),
    run_thesis('our_problems/problem12', running,
               0.0, running_thesis[1]),

    run_satisfiability('our_problems/problem2',
                       repair, 0.0),
    run_satisfiability('our_problems/problem3',
                       repair, 0.5),
    run_satisfiability('our_problems/problem4',
                       repair, 1.0),
    run_comparison('our_problems/problem5', repair,
                   0.0, 0.5, LogicFunctions.IMPLIES),
    run_comparison('our_problems/problem6', repair,
                   0.5, 1.0, LogicFunctions.IMPLIES),
    run_comparison('our_problems/problem8', repair,
                   0.0, 0.5, LogicFunctions.EQUIVALENT),
    run_comparison('our_problems/problem9', repair,
                   0.5, 1.0, LogicFunctions.EQUIVALENT),
    run_thesis('our_problems/problem13', repair,
               0.5, repair_thesis[0]),
    run_thesis('our_problems/problem14', repair,
               0.5, repair_thesis[1]),
    run_thesis('our_problems/problem15', repair,
               1.0, repair_thesis[0]),
    run_thesis('our_problems/problem16', repair,
               1.0, repair_thesis[1]),
]

Seq3(1]register_request, Loop(2]l_s_0, Seq2(3]And2(4]a2_s_0, check_ticket, Xor2(5]x2_s_0, examine_casually, examine_thoroughly, x2_e_0[5), a2_e_0[4), decide[3), reinitiate_request[2), Xor2(2]x2_s_1, pay_compensation, reject_request, x2_e_1[2)[1)
ini: register_request
fin: x2_e_1
Seq3(1]register_request, Loop(2]l_s_0, Seq2(3]And2(4]a2_s_0, check_ticket, Xor2(5]x2_s_0, examine_casually, examine_thoroughly, x2_e_0[5), a2_e_0[4), decide[3), reinitiate_request[2), Xor2(2]x2_s_1, pay_compensation, reject_request, x2_e_1[2)[1)
ini: register_request
fin: x2_e_1
Seq3(1]register_request, Loop(2]l_s_0, Seq2(3]And2(4]a2_s_0, check_ticket, Xor2(5]x2_s_0, examine_casually, examine_thoroughly, x2_e_0[5), a2_e_0[4), decide[3), reinitiate_request[2), Xor2(2]x2_s_1, pay_compensation, reject_request, x2_e_1[2)[1)
ini: register_request
fin: x2_e_1
Seq5(1]register, analyze_defect, And2(2]a2_s_0, Xor2(3]x2_s_0, tau_0, inform_user, x2_e_0[3), And2(3]a2_s_1, Xor2(4]x2_s_1, tau_1, Loop(5]l_s_0, test_repair, ta

### Readable output

In [13]:
for r in our_results:
    print(r[0])
    print('  Vampire:')
    for i in r[1].split('%')[1:]:
        print(f'    %{i}')
    print('  Eprover:')
    for i in r[2].split('#')[1:]:
        print(f'    %{i}')
    print('\n')

our_problems/problem1
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status Satisfiable


our_problems/problem11
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_thesis 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Theorem


our_problems/problem12
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_thesis 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Theorem


our_problems/problem2
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status Satisfiable


our_problems/problem3
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status Satisfiable


our_problems/problem4
  Vampire:
    % Refutation

### Csv

In [ ]:
our_csv = '\n'.join([f'{r[0]},{r[1]},{r[2]}' for r in our_results])
with open('our_problems/our_results.csv', 'w') as f:
    f.write(our_csv)

# Big problems

## Logs

In [15]:
hospital = pm4py.read_xes('Data/Hospital Billing - Event Log.xes')

parsing log, completed traces :: 100%|██████████| 100000/100000 [00:12<00:00, 7771.71it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=T

In [16]:
bpic = pm4py.read_xes('Data/bpic2012.xes')

parsing log, completed traces :: 100%|██████████| 13087/13087 [00:07<00:00, 1751.56it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=Tru

In [17]:
artificial = pm4py.read_xes('Data/log_3_1732138120.xes')

parsing log, completed traces :: 100%|██████████| 30369/30369 [00:11<00:00, 2663.08it/s]
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)


## Theorems

In [18]:
hospital_theorems = [
    '''
fof(thesis, conjecture, (
    ?[X]: new(X) => ?[Y]: join_pat(Y)
)).
''',
    '''
fof(thesis, conjecture, (
    ![X]: new(X) => ?[Y]: join_pat(Y)
)).
'''
]

bpic_theorems = [
    '''
fof(thesis, conjecture, (
    ![X]: ~(w_afhandelen_leads(X) & a_preaccepted(X))
)).
''',
    '''
fof(thesis, conjecture, (
    ![X]: ~(a_submitted(X) => a_partlysubmitted(X))
)).
'''
]

artificial_theorems = [
    '''
fof(thesis, conjecture, (
    ![X]: (h(X) & ~k(X))
)).
''',
    '''
fof(thesis, conjecture, (
    ![X]: ~(n(X) & g(X))
)).
'''
]

## Code

In [19]:
big_results = [
    run_satisfiability('big_problems/hospital/satisfiability_0', hospital, 0.0),
    run_satisfiability('big_problems/hospital/satisfiability_25', hospital, 0.25),

    run_comparison('big_problems/hospital/comparison_0_25_equivalent', hospital, 0.0, 0.25, LogicFunctions.EQUIVALENT),
    run_comparison('big_problems/hospital/comparison_0_25_implies', hospital, 0.0, 0.25, LogicFunctions.IMPLIES),

    run_thesis('big_problems/hospital/thesis_0', hospital, 0.25, hospital_theorems[0]),
    run_thesis('big_problems/hospital/thesis_1', hospital, 0.25, hospital_theorems[1]),


    run_satisfiability('big_problems/bpic/satisfiability_0', bpic, 0.0),
    run_satisfiability('big_problems/bpic/satisfiability_25', bpic, 0.25),

    run_comparison('big_problems/bpic/comparison_0_25_equivalent', bpic, 0.0, 0.25, LogicFunctions.EQUIVALENT),
    run_comparison('big_problems/bpic/comparison_0_25_implies', bpic, 0.0, 0.25, LogicFunctions.IMPLIES),

    run_thesis('big_problems/bpic/thesis_0', bpic, 0.25, bpic_theorems[0]),
    run_thesis('big_problems/bpic/thesis_1', bpic, 0.25, bpic_theorems[1]),


    run_satisfiability('big_problems/artificial/satisfiability_0', artificial, 0.0),
    run_satisfiability('big_problems/artificial/satisfiability_25', artificial, 0.25),

    run_comparison('big_problems/artificial/comparison_0_25_equivalent', artificial, 0.0, 0.25, LogicFunctions.EQUIVALENT),
    run_comparison('big_problems/artificial/comparison_0_25_implies', artificial, 0.0, 0.25, LogicFunctions.IMPLIES),

    run_thesis('big_problems/artificial/thesis_0', artificial, 0.25, artificial_theorems[0]),
    run_thesis('big_problems/artificial/thesis_1', artificial, 0.25, artificial_theorems[1]),
]

Seq2(1]Loop(2]l_s_0, new, tau_0[2), Xor2(2]x2_s_0, tau_1, And2(3]a2_s_0, Xor2(4]x2_s_1, tau_2, Loop(5]l_s_1, billed, tau_3[5), x2_e_1[4), Xor2(4]x2_s_2, tau_4, Seq2(5]And2(6]a2_s_1, Xor2(7]x2_s_3, tau_5, Loop(8]l_s_2, join_pat, tau_6[8), x2_e_3[7), Xor2(7]x2_s_4, tau_7, Seq2(8]Xor2(9]x2_s_5, tau_8, Seq2(10]Loop(11]l_s_3, Xor3(12]x3_s_6, change_end, Seq2(13]Xor2(14]x2_s_7, tau_9, set_status, x2_e_7[14), Xor2(14]x2_s_8, tau_10, And2(15]a2_s_2, Xor2(16]x2_s_9, tau_11, Loop(17]l_s_4, code_error, tau_12[17), x2_e_9[16), Seq3(16]Xor2(17]x2_s_10, tau_13, Loop(18]l_s_5, release, tau_14[18), x2_e_10[17), Xor2(17]x2_s_11, tau_15, Xor2(18]x2_s_12, manual, code_ok, x2_e_12[18), x2_e_11[17), Xor2(17]x2_s_13, tau_16, And2(18]a2_s_3, Xor2(19]x2_s_14, tau_17, Loop(20]l_s_6, code_nok, tau_18[20), x2_e_14[19), Xor2(19]x2_s_15, tau_19, Seq2(20]Xor2(21]x2_s_16, tau_20, Loop(22]l_s_7, Seq2(23]storno, Xor2(24]x2_s_17, tau_21, Loop(25]l_s_8, reject, tau_22[25), x2_e_17[24)[23), tau_23[22), x2_e_16[21), Xor2(

## Readable output

In [20]:
for r in big_results:
    print(r[0])
    print('  Vampire:')
    for i in r[1].split('%')[1:]:
        print(f'    %{i}')
    print('  Eprover:')
    for i in r[2].split('#')[1:]:
        print(f'    %{i}')
    print('\n')

big_problems/hospital/satisfiability_0
  Vampire:
    % SZS status Satisfiable for tptp 
    % Termination reason: Satisfiable
  Eprover:
    % No proof found! 
    % SZS status Satisfiable


big_problems/hospital/satisfiability_25
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Unsatisfiable for tptp 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status Unsatisfiable


big_problems/hospital/comparison_0_25_equivalent
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_EQUIVALENT 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status ContradictoryAxioms


big_problems/hospital/comparison_0_25_implies
  Vampire:
    % Refutation found. Thanks to Tanya! 
    % SZS status Theorem for tptp_IMPLIES 
    % Termination reason: Refutation
  Eprover:
    % Proof found! 
    % SZS status ContradictoryAxioms


big_problems/hospital/thesis_0
  Vampire:
    % Refutation found. Tha

## Csv

In [21]:
big_csv = '\n'.join([f'{r[0]},{r[1]},{r[2]}' for r in big_results])
with open('big_problems/big_results.csv', 'w') as f:
    f.write(big_csv)